In [1]:
from glob import glob
import numpy as np
from keras import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler, LabelEncoder
data_path = "/Users/soltan/Programs/kaggle/raw_data/train/train/*"

import librosa
def extract_features(file_name):
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast')

    # Generate Mel-frequency cepstral coefficients (MFCCs) from a time series
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)

    # Generates a Short-time Fourier transform (STFT) to use in the chroma_stft
    stft = np.abs(librosa.stft(X))

    # Computes a chromagram from a waveform or power spectrogram.
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)

    # Computes a mel-scaled spectrogram.
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)

    # Computes spectral contrast
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)

    # Computes the tonal centroid features (tonnetz)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),sr=sample_rate).T,axis=0)

    return np.concatenate([mfccs, chroma, mel, contrast, tonnetz])

Using TensorFlow backend.


In [2]:
from tqdm import tqdm
data_dir = np.array(glob(data_path))
features, labels = [], []
for file in tqdm(data_dir):
    file_name = file.split("/")[-1]
    file_name = file.split(".")[0]
    name, label = file_name.split("-")[0], file_name.split("-")[1]
    features.append(extract_features(file))
    labels.append(label)



100%|██████████| 9000/9000 [19:06<00:00,  7.85it/s]  


In [3]:
from sklearn.model_selection import train_test_split
inputs_train, inputs_test, targets_train, targets_test = train_test_split(features, labels, test_size=0.2)

In [4]:

ss = StandardScaler()
X_train = ss.fit_transform(inputs_train)
X_val = ss.transform(inputs_test)


lb = LabelEncoder()
y_train = to_categorical(lb.fit_transform(targets_train))
y_val = to_categorical(lb.fit_transform(targets_test))


In [11]:

model = Sequential()

model.add(Dense(193, input_shape=(193,), activation = 'relu'))
model.add(Dropout(0.1))

model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.25))

model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(2, activation = 'softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto')
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 193)               37442     
_________________________________________________________________
dropout_7 (Dropout)          (None, 193)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               24832     
_________________________________________________________________
dropout_8 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 128)               16512     
_________________________________________________________________
dropout_9 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 2)                

In [12]:
history = model.fit(X_train, y_train, batch_size=1000, epochs=30,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stop])

Train on 7200 samples, validate on 1800 samples
Epoch 1/30
7200/7200 [==============================] - 2s 223us/step - loss: 0.7460 - accuracy: 0.5254 - val_loss: 0.6749 - val_accuracy: 0.5844
Epoch 2/30
7200/7200 [==============================] - 0s 41us/step - loss: 0.6954 - accuracy: 0.5592 - val_loss: 0.6620 - val_accuracy: 0.6111
Epoch 3/30
7200/7200 [==============================] - 0s 42us/step

In [ ]:
test_path = "/Users/soltan/Programs/kaggle/raw_data/test/test/*"
from tqdm import tqdm
test_dir = np.array(glob(test_path))

fout = open("../submission.txt", "w")
fout.write("name,label\n")
for file in tqdm(test_dir):
    name = file.split("/")[-1]
    ft = extract_features(file)
    ft = ss.transform([ft])
    pred = model.predict_classes([ft])[0]
    fout.write("{},{}\n".format(name, pred))

fout.close()



 62%|██████▏   | 1867/3000 [05:35<03:31,  5.36it/s]